In [1]:
#1
#%matplotlib inline
%matplotlib notebook
from sklearn.utils import shuffle
import pandas as pd
import csv
import numpy as np
import time 
# for database operations
from haversine import haversine
import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_similarity_score 

In [2]:
#2

def ST_DBSCAN(points,max_distance,MinPts,dayRadius):
    global visited 
    visited = []
    noise = []
    c_neighbors = []
    cluster_id = 0
    clusters = []
    in_cluster = []
    for p in points: 
        if p not in visited:
            # neighbor_points = []
            visited.append(p)
            NeighborPts = regionQuery(p,points,max_distance,dayRadius)
            if len(NeighborPts) < MinPts:
                noise.append(p)
            else:
                cluster_id = cluster_id + 1
                g = expandCluster(p,NeighborPts,max_distance,MinPts,in_cluster,cluster_id)
                clusters.append(g)
                c_neighbors=NeighborPts+[cluster_id]
                #print c_neighbors
                c_neighbors.append(g)
    return clusters, c_neighbors

#return len(NeighborPts)

def expandCluster(p,NeighborPts,max_distance,MinPts,in_cluster,cluster_id):
    in_cluster.append(p)
    cluster = []
    t_p = list(p) + [cluster_id]
    cluster.append(t_p)
    for point in NeighborPts:
        if point not in visited:
            visited.append(point)
            new_neighbors = regionQuery(point,points,max_distance,dayRadius)
            if len(new_neighbors) >= MinPts: 
                new_neighbors.append(NeighborPts)
            if point not in in_cluster:
                 t_point = list(point) + [cluster_id]
                 in_cluster.append(point)
                 cluster.append(t_point)
    return cluster


def regionQuery(p,points,max_distance,dayRadius):
    neighbor_points = []
    loc1 = (p[0],p[1])
    for j in points:
        if j != p:
            # print 'P is %s and j is %s' % (p[0],j[0])
            if abs(p[2]-j[2]) < dayRadius:
                loc2 = (j[0],j[1])
                dist = haversine(loc1, loc2)
                if dist <= max_distance:
                    neighbor_points.append(j)
    neighbor_points.append(p) 
    return neighbor_points   


In [3]:
#3
max_distance = 80
MaxPts = 10
dayRadius = 10

df = pd.DataFrame.from_csv('D:/Nanohub/results/data extract from database/in_STDB_2014_2.csv', sep ='\t', encoding = 'UTF-8', header = None)
df.columns = ['ip','resource_id','datetime','latitude','longitude','child_id','title', 'ip_domainname', 'ip_country', 'uidNumber']
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(by = 'resource_id', axis = 0, ascending = True, na_position ='first')
df_no_dups = df.drop_duplicates(subset = ['ip','resource_id'], keep = 'first')
doy = []
for date in df['datetime']:
    doy.append(date.dayofyear)
df['DoY'] = doy

df_10 = df[df.groupby('resource_id').cumcount(ascending=False) >= 10]
distinct_resources = list(df_10['resource_id'].unique())

In [4]:
#4
uniq_ip_2015 = df_10['ip'].unique()
uniq_res_2015 = df_10['resource_id'].unique()
sims = np.zeros((len(uniq_res_2015),len(uniq_ip_2015)))
uniq_ip_2015.sort()
   

In [12]:
sims.shape

(1505, 11211)

In [13]:
#5 STDBSCAN call
for l in distinct_resources: #range(21095,22000):
    df_filtered = df_10[df_10['resource_id'] == l]
    df_filtered =df_filtered[df_filtered['latitude'] > 0]
    tups = df_filtered.set_index(['latitude'])[['longitude','DoY','resource_id','ip']].to_records().tolist()
    points = tups
    core, neigh = ST_DBSCAN(points,max_distance,MaxPts,dayRadius)
    
    fname = 'D:/Nanohub/results/GIS6/' + str(l) +'.csv'
    
    f = open(fname, 'wt')
    try:
        writer = csv.writer(f, lineterminator = '\n')
        #writer.writerow( ('Latitude', 'Longitude', 'DoY' , 'res_id' , 'cluster_id') )

        for j in range(len(core)):
            for i in range(len(core[j])):
                    writer.writerow( core[j][i] )
    finally:
        f.close()

In [5]:
#6 similarity matrix
from pathlib import Path


for k in range(len(uniq_res_2015)):
    fstr = Path('D:/Nanohub/results/GIS6/' + str(uniq_res_2015[k]) + '.csv')
    if fstr.is_file():
        df_open = pd.DataFrame.from_csv(fstr, sep =',', encoding = 'UTF-8', header = None)
        df_open.columns = ['longitude','DOY','resource_id','ip','cluster_id']
        ips_of_file = df_open['ip'].unique()
        ips_of_file.sort()
        if len(df_open) > 0:
            for j in range(len(ips_of_file)):
                for i in range(len(uniq_ip_2015)):
                    if ips_of_file[j] == uniq_ip_2015[i]:
                        sims[k][i] = 1
        

In [6]:
print(sims[1][90:200])
print(uniq_res_2015[1])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
8


In [ ]:
#7 incorrect

for s1 in range(len(sims)):
    #print ('s1',uniq_res_2015[s1])
    arr1 = []
    for s2 in range(len(sims)):
        if s2 != s1:
            if jaccard_similarity_score(sims[s1],sims[s2]) > 0.8:
                arr1.append(uniq_res_2015[s2])
    print('---------------------------------------------------')            
    print (len(arr1))

In [31]:
fjacc = open('D:/Nanohub/results/GIS6/jacc6.csv','w')
for s1 in range(len(sims)):
    a1 = sims[s1]
    arr1 = []
    jacc = -1
    for s2 in range(len(sims)):
        if s1!=s2:
            a2 = sims[s2]
            inter_1 = 0
            all_1 = 0
            for i in range(len(a1)):
                all_1 = all_1 + a1[i] + a2[i]
                if a1[i] == a2[i] and a1[i] == 1:
                    inter_1 += 1
                if all_1-inter_1 >0:
                    jacc = float(float(inter_1) / float(all_1-inter_1))
        
            if jacc > 0.5 and jacc != -1:
                arr1.append(uniq_res_2015[s2])
    if len(arr1)!=0:
        print (uniq_res_2015[s1],arr1)
    
    '''if len(arr1) > 0:
       
        writer = csv.writer(fjacc, lineterminator = '\n')
        writer.writerow(arr1 )'''        
    

125 [126, 131]
126 [125, 130, 131]
130 [126, 131, 3585]
131 [125, 126, 130]
135 [924, 988]
222 [9525]
416 [4962]
924 [135, 988]
988 [135, 924]
1514 [22879]
1840 [1845, 5307, 5309, 12707]
1845 [1840, 5307, 5309, 12707]
3059 [13159, 17282, 22329]
3585 [130]
4754 [4757]
4757 [4754]
4962 [416]
5307 [1840, 1845, 5309, 12707]
5309 [1840, 1845, 5307, 12707]
6806 [7486]
7320 [21880]
7486 [6806]
7780 [7827, 12456]
7827 [7780, 12456]
8742 [10165, 10226]
9109 [17591]
9162 [9167]
9167 [9162]
9208 [9231, 9525]
9231 [9208, 9525]
9525 [222, 9208, 9231]
10165 [8742]
10226 [8742]
12453 [12456, 12457]
12456 [7780, 7827, 12453, 12457]
12457 [12453, 12456]
12707 [1840, 1845, 5307, 5309]
13159 [3059, 17282, 19169, 22329, 23261]
13554 [16362]
14264 [16221]
16221 [14264]
16362 [13554]
16511 [16546]
16545 [16546]
16546 [16511, 16545]
16548 [16608]
16608 [16548, 16774, 16789]
16703 [19299]
16774 [16608, 16789]
16789 [16608, 16774]
16887 [16895, 16896]
16895 [16887, 16896, 16920, 16965, 17028, 17293, 17295]
168